In [ ]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install accelerate
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.6/302.6 kB 6.3 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

In [ ]:

import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from huggingface_hub import HfFolder, Repository
from sklearn.metrics import cohen_kappa_score
from datasets import Dataset, get_dataset_config_names
import polars as pl
from sklearn.preprocessing import KBinsDiscretizer
import numpy as np
from transformers import (AutoTokenizer,
                          TrainingArguments,
                          DataCollatorWithPadding,
                          Trainer,
                          AutoModelForSequenceClassification,
                           EarlyStoppingCallback
                          )
import regex as re
from sklearn.metrics import accuracy_score, f1_score
import numpy as np
from google.colab import userdata
import ctypes, torch, gc, yaml,  joblib, spacy, multiprocessing, os, random


In [ ]:
key_ = userdata.get('HF_TOKEN')
cuda_available = torch.cuda.is_available()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
train_path = '/content/drive/MyDrive/essay/data/train.csv'
libc = ctypes.CDLL("libc.so.6")

cuda


In [ ]:
args_no = 3
args = f'/content/drive/MyDrive/essay/args/args_{args_no}.yaml'

with open(args, 'r') as file:
  config = yaml.safe_load(file)


In [ ]:

def clear_memory():
    libc.malloc_trim(0)
    torch.cuda.empty_cache()
    gc.collect()



In [ ]:
lbl_mapping = { j: i for i, j in enumerate(config['lbl'])}


df_full = (pl.scan_csv(train_path)
        .with_columns(
            score = pl.col('score')
            .replace(lbl_mapping)
        )
      )


In [ ]:
def initialize_seeds(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

initialize_seeds(config['seed'])

In [ ]:


class text_prep():
  def __init__(self,
               df,
               tokenizer,
               max_length = config['max_length'],
               batch_size = config['batch_size'],
               stride = config['stride']
               ):

    self.set_df(df)
    self.tokenizer = tokenizer
    self.max_length = max_length
    self.batch_size  = batch_size
    self.stride = stride

  def tokenize_function(self, batch):
    try:
        input_ids = []
        essay_id = []
        attention_mask = []
        full_text = []
        score = []
        txts = [self.preprocess_text(i) for i in batch['full_text']]

        for count_id , txt in enumerate(txts):
          tokenized = self.tokenizer(txt,
                                    max_length=self.max_length,
                                    return_overflowing_tokens = True,
                                    stride = self.stride,
                                    truncation=True
                                    )


          for count, input in enumerate(tokenized['input_ids']):
            full_text.append(txt)
            score.append(batch['score'][count_id])
            essay_id.append(batch['essay_id'][count_id])
            input_ids.append(input)
            attention_mask.append(tokenized['attention_mask'][count])

        return {
            'essay_id': essay_id,
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'full_text': full_text,
            'score': score
        }
    except Exception as e:
        print(f"Error tokenizing batch: {batch}")
        raise e

  def preprocess_text(self, text):
    text = text.replace('\xa0', ' ')
    text = re.sub(r"[^a-zA-Z0-9\s,.'!?;:-]", '', text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

  def set_df(self, val):
    if isinstance(val, Dataset):
      self.df = val
    elif isinstance(val, pl.DataFrame):
      arrow_table = val.to_arrow()
      self.df = Dataset(arrow_table)
    else:
      raise TypeError("Unsupported data")



  def fit_transform(self):
     return (self.df.map(self.tokenize_function,
                         num_proc=multiprocessing.cpu_count() -1,
                         batched = True
                         )
                    .rename_column('score', 'labels')
     )


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(config['model_ckpt_berta'])
def token_df(tokenizer, df_full):
  df_sample = df_full.collect()
  tp = text_prep(df_sample, tokenizer = tokenizer)
  return tp.fit_transform()


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

In [ ]:


def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    qwk = cohen_kappa_score(p.label_ids, preds, weights="quadratic")
    return {
        'qwk': float(qwk),
    }


In [ ]:
df = token_df(tokenizer, df_full)



/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Map (num_proc=11):   0%|          | 0/17307 [00:00<?, ? examples/s]

In [ ]:

essay_id = list(set(df['essay_id']))

df_pd = (
        df_full
        .filter(pl.col('essay_id').is_in(essay_id))
        .collect()
        .to_pandas()
)


df_dic = {}
stratified_kfold = StratifiedKFold(n_splits=config['num_folds'], shuffle=True, random_state=config['seed'])
for fold, (train_index, test_index) in enumerate(stratified_kfold.split(df_pd, df_pd['score'])):
    train_index, val_index = train_test_split(train_index, test_size=0.2, random_state=config['seed'])
    df_dic[fold] = {
      'train': df.filter(lambda f: f['essay_id'] in df_pd.iloc[train_index].loc[:, 'essay_id'].to_list()),
      'vali': df.filter(lambda f: f['essay_id'] in df_pd.iloc[val_index].loc[:, 'essay_id'].to_list()),
      'test':  df.filter(lambda f: f['essay_id'] in df_pd.iloc[test_index].loc[:, 'essay_id'].to_list())
    }



Filter:   0%|          | 0/22212 [00:00<?, ? examples/s]

Filter:   0%|          | 0/22212 [00:00<?, ? examples/s]

Filter:   0%|          | 0/22212 [00:00<?, ? examples/s]

Filter:   0%|          | 0/22212 [00:00<?, ? examples/s]

Filter:   0%|          | 0/22212 [00:00<?, ? examples/s]

...

Filter:   0%|          | 0/22212 [00:00<?, ? examples/s]

Filter:   0%|          | 0/22212 [00:00<?, ? examples/s]

Filter:   0%|          | 0/22212 [00:00<?, ? examples/s]

Filter:   0%|          | 0/22212 [00:00<?, ? examples/s]

Filter:   0%|          | 0/22212 [00:00<?, ? examples/s]

In [ ]:
df_dic[0]


{'train': Dataset({
     features: ['essay_id', 'full_text', 'labels', 'input_ids', 'attention_mask'],
     num_rows: 14220
 }),
 'vali': Dataset({
     features: ['essay_id', 'full_text', 'labels', 'input_ids', 'attention_mask'],
     num_rows: 3564
 }),
 'test': Dataset({
     features: ['essay_id', 'full_text', 'labels', 'input_ids', 'attention_mask'],
     num_rows: 4428
 })}

In [ ]:
data_collator = DataCollatorWithPadding(tokenizer, pad_to_multiple_of=config['pad_to_multiple_of'])


def do_trainer(train, vali, fold, args):
  model = (AutoModelForSequenceClassification.from_pretrained(config['model_ckpt_berta'],
                                                            num_labels=len(config['lbl']))
         .to(device)
         )

  training_args = TrainingArguments(
      output_dir=f"{config['model_dir']}{args}_roberta_{fold}",
      learning_rate=1e-5,
      num_train_epochs=config['epoch'],
      per_device_train_batch_size=config['batch_size'],
      warmup_steps=config['warmup_steps'],
      per_device_eval_batch_size=config['batch_size'],
      weight_decay=0.01,
      save_total_limit=1,
      logging_steps=len(train)//(config['batch_size'] * config['gradient_accumulation_steps']) ,
      evaluation_strategy="epoch",
      save_strategy="epoch",
      metric_for_best_model="qwk",
      greater_is_better=True,
      load_best_model_at_end=True,
      hub_token=key_,
      push_to_hub=True,
      lr_scheduler_type='cosine',
      fp16=True,
      gradient_accumulation_steps = config['gradient_accumulation_steps']
  )

  trainer = Trainer(
      model=model,
      args=training_args,
      train_dataset=train,
      eval_dataset= vali,
      tokenizer=tokenizer,
      data_collator=data_collator,
      compute_metrics=compute_metrics,
      callbacks=[EarlyStoppingCallback(config['early_stopping_patience'])]
  )
  return trainer



In [ ]:
# clear_memory()


In [ ]:

for fold, data in df_dic.items():
  train = data['train']
  vali = data['vali']
  test = data['test']

  train = train.remove_columns(['essay_id','full_text' ])
  vali = vali.remove_columns(['essay_id','full_text'])
  trainer_ = do_trainer(train, vali, fold, args_no)
  print(f'fold_{fold}')
  try:
      trainer_.train()
  except Exception as e:
      print("Error during training: %s", e)


  try:
      trainer_.evaluate()
  except Exception as e:
      print("Error during evaluation: %s", e)

  clear_memory()


model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


fold_0


Epoch,Training Loss,Validation Loss,Qwk
0,1.361900,1.096990,0.632210
2,0.864800,0.929524,0.759642
4,0.693900,1.161541,0.750064


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


...

fold_3


Epoch,Training Loss,Validation Loss,Qwk
1,1.372000,1.094822,0.690695
2,0.991500,0.925787,0.746963
3,0.880400,0.889719,0.789441
4,0.797900,0.895531,0.784007
5,0.713000,0.942066,0.802720
6,0.630500,1.040864,0.792039
7,0.546000,1.068723,0.797789


Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at FacebookAI/roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


fold_4


Epoch,Training Loss,Validation Loss,Qwk
0,1.354100,1.074934,0.679041
2,0.870600,0.905548,0.791514
4,0.694000,1.039868,0.766883
